In [1]:
# Environment: conda python 3.7

from DijkstraSolver import DijkstraSolver, singleAccessibilitySolve
from BasicSolver import BasicSolver
basicSolver = BasicSolver()

walkingDistanceLimit = 700
timeDeltaLimit = 180 * 60
walkingSpeed = 1.4
isRealTime = True

In [2]:
# Calculating OD matrix
startLocation = {
    "stop_lat": 39.963596, 
    "stop_lon": -83.000944
}
timestamp = 1573942335
args = {
    "timestamp": timestamp, 
    "walkingDistanceLimit": walkingDistanceLimit, 
    "timeDeltaLimit": timeDeltaLimit, 
    "walkingSpeed": walkingSpeed, 
    "isRealTime": isRealTime, 
}

solver, results = singleAccessibilitySolve(args, startLocation)


------ Retrospective Timetable Routing Running... ------


  0%|          | 0/3082 [00:00<?, ?it/s]

------ Retrospective Timetable Routing Finished... ------
------ Scheduled Timetable Routing Running... ------


  0%|          | 0/3082 [00:00<?, ?it/s]

------ Scheduled Timetable Routing Finished... ------


In [22]:
# Produce buffers
from shapely.geometry import Point, mapping
import json
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
timeBudgetList = list(range(5,120,5))

columnsHead = ["lat", "lon"]
for timeBudget in timeBudgetList:
    columnsHead.append("R_" + str(timeBudget))

parameterList = []
for receivingStopID, ODRecord in (results.items()):
    insertRecord = [float(ODRecord["stop_lat"]), float(ODRecord["stop_lon"])]
    for timeBudget in timeBudgetList: 
        radius = int(max(min((timeBudget * 60 - ODRecord["timeSC"]) * walkingSpeed, walkingDistanceLimit), 0))
        insertRecord.append(radius)
    parameterList.append(insertRecord)

geojsonList = {}
df = pd.DataFrame(parameterList, columns = columnsHead)
geometry = [Point(xy) for xy in zip(df.lon, df.lat)]
df = df.drop(['lon', 'lat'], axis=1)
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)
gdf_p = gdf.to_crs("EPSG:32123")
for timeBudget in tqdm(timeBudgetList):
    buffers = gdf_p.buffer(distance=df["R_" + str(timeBudget)])
    buffers_p = buffers.to_crs("EPSG:4326")
    buffer_gdf = gpd.GeoDataFrame(geometry=buffers_p)
    buffer_gdf = buffer_gdf.dissolve()
    geo_json = json.loads(buffer_gdf.to_json())
    geojsonLayer = GeoJSON(
        data=geo_json
    )
    geojsonList["R_" + str(timeBudget)] = geojsonLayer

# Visualizae PPA
from ipyleaflet import Map, basemaps, Circle, WidgetControl, GeoJSON, LayerGroup
from ipywidgets import IntSlider, ColorPicker, jslink

defaultTimeBudget = 30
timeBudgetBackup = defaultTimeBudget

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(39.963596, -83.000944), zoom=11)
slider = IntSlider(min=5, max=120, step=5, value=defaultTimeBudget, continuous_update=True)
slider.observe(onChange, names='value')
m.add_layer(geojsonList["R_" + str(defaultTimeBudget)])

def onChange(e):
    timeBudget = int(e["new"])
    oldTimeBudget = int(e["old"])
    m.remove_layer(geojsonList["R_" + str(oldTimeBudget)])
    m.add_layer(geojsonList["R_" + str(timeBudget)])


widget_control1 = WidgetControl(widget=slider, position='topright')
m.add_control(widget_control1)

m


  0%|          | 0/23 [00:00<?, ?it/s]

Map(center=[39.963596, -83.000944], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

{'name': 'value', 'old': 30, 'new': 35, 'owner': IntSlider(value=35, max=120, min=5, step=5), 'type': 'change'}
{'name': 'value', 'old': 35, 'new': 40, 'owner': IntSlider(value=40, max=120, min=5, step=5), 'type': 'change'}
{'name': 'value', 'old': 40, 'new': 45, 'owner': IntSlider(value=45, max=120, min=5, step=5), 'type': 'change'}
{'name': 'value', 'old': 45, 'new': 50, 'owner': IntSlider(value=50, max=120, min=5, step=5), 'type': 'change'}
{'name': 'value', 'old': 50, 'new': 55, 'owner': IntSlider(value=55, max=120, min=5, step=5), 'type': 'change'}
{'name': 'value', 'old': 55, 'new': 60, 'owner': IntSlider(value=60, max=120, min=5, step=5), 'type': 'change'}
{'name': 'value', 'old': 60, 'new': 65, 'owner': IntSlider(value=65, max=120, min=5, step=5), 'type': 'change'}
{'name': 'value', 'old': 65, 'new': 70, 'owner': IntSlider(value=70, max=120, min=5, step=5), 'type': 'change'}
{'name': 'value', 'old': 70, 'new': 75, 'owner': IntSlider(value=75, max=120, min=5, step=5), 'type': 'c